# RevDEQ実験 - Loss 25まで下がることを確認

このノートブックでは、RevDEQモデルを学習し、lossが25程度まで下がることを確認します。


## 2. リポジトリのクローン

In [ ]:
import os
import sys

# GitHubリポジトリをクローン
!git clone https://github.com/kaede-mori/revdeq-pytorch.git
%cd revdeq-pytorch

# パスを追加
sys.path.insert(0, os.getcwd())

print(f"Current directory: {os.getcwd()}")
print(f"Repository cloned successfully!")

## 3. モジュールのインポート

In [ ]:
from revdeq import RevDEQ, RevDEQConfig
from train import prepare_dataset, RevDEQDataset
from transformers import (
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling
)
from datasets import load_dataset
import matplotlib.pyplot as plt
import numpy as np

print("All modules imported successfully!")

## 4. モデルの設定（loss 25を目標に最適化）

In [ ]:
# 必要なライブラリのインストール
%pip install -q torch transformers datasets accelerate tqdm wandb pyyaml tensorboard

# GPU確認
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")


## 5. データセットの準備


In [ ]:
# WikiText-2データセットをロード
print("Loading WikiText-2 dataset...")
texts = prepare_dataset("wikitext", "wikitext-2-raw-v1", max_texts=None)  # 全データを使用

print(f"Loaded {len(texts)} text examples")
print(f"Sample text: {texts[0][:100]}...")

# データセットの作成
train_dataset = RevDEQDataset(texts, tokenizer, max_length=model_config.max_position_embeddings)
print(f"Dataset size: {len(train_dataset)} examples")


## 6. 学習設定（loss 25を目指す最適化）


In [ ]:
# Loss 25を目指すための学習設定
training_args = TrainingArguments(
    output_dir="./checkpoints",
    num_train_epochs=5,  # 複数エポックで学習
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,  # 実質的なバッチサイズ = 4 * 8 = 32
    learning_rate=3e-4,  # 少し高めの学習率で効率的に学習
    weight_decay=0.01,
    warmup_steps=500,
    logging_steps=50,  # 50ステップごとにログ出力
    save_steps=1000,
    save_total_limit=2,
    fp16=torch.cuda.is_available(),  # GPU使用時はFP16で高速化
    eval_strategy="no",
    save_strategy="no",  # 手動で保存
    report_to=[],  # TensorBoardは使わない（Colabで複雑になるため）
    remove_unused_columns=False,
    dataloader_num_workers=2,
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

print("Training arguments configured!")


## 7. カスタムTrainer（loss追跡用）


In [ ]:
# Loss履歴を追跡するカスタムTrainer
class RevDEQTrainerWithHistory(Trainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.loss_history = []
        self.step_history = []
    
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        
        if isinstance(outputs, dict):
            loss = outputs.get("loss")
            if return_outputs:
                return loss, outputs
            return loss
        elif isinstance(outputs, tuple):
            logits, loss = outputs
            if return_outputs:
                return loss, {"logits": logits}
            return loss
        else:
            if return_outputs:
                return None, outputs
            return None
    
    def log(self, logs):
        # Lossを記録
        if "loss" in logs:
            self.loss_history.append(logs["loss"])
            self.step_history.append(logs.get("step", len(self.loss_history)))
            
            # 25以下になったら通知
            if logs["loss"] <= 25.0:
                print(f"\n🎉 Loss reached target! Current loss: {logs['loss']:.4f}")
        
        super().log(logs)

trainer = RevDEQTrainerWithHistory(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=data_collator,
)

print("Trainer initialized with loss tracking!")


## 8. 学習の実行


In [ ]:
# 学習開始
print("=" * 60)
print("Starting training...")
print(f"Target: Loss <= 25.0")
print("=" * 60)

train_result = trainer.train()

print("\n" + "=" * 60)
print("Training completed!")
print(f"Final training loss: {train_result.training_loss:.4f}")
print("=" * 60)


## 9. Lossの可視化


In [ ]:
# Loss履歴の可視化
if len(trainer.loss_history) > 0:
    plt.figure(figsize=(12, 6))
    
    # Loss曲線
    steps = trainer.step_history if trainer.step_history else range(len(trainer.loss_history))
    plt.plot(steps, trainer.loss_history, label='Training Loss', linewidth=2)
    
    # 目標線（Loss = 25）
    plt.axhline(y=25.0, color='r', linestyle='--', label='Target (Loss = 25)', linewidth=2)
    
    plt.xlabel('Step', fontsize=12)
    plt.ylabel('Loss', fontsize=12)
    plt.title('RevDEQ Training Loss Progress', fontsize=14, fontweight='bold')
    plt.legend(fontsize=11)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    
    # 最終的なlossを表示
    final_loss = trainer.loss_history[-1]
    min_loss = min(trainer.loss_history)
    
    plt.text(0.02, 0.98, 
             f'Final Loss: {final_loss:.4f}\n' +
             f'Min Loss: {min_loss:.4f}\n' +
             f'Target: ≤25.0\n' +
             f'Status: {"✅ Achieved!" if min_loss <= 25.0 else "⏳ Not yet"}',
             transform=plt.gca().transAxes,
             fontsize=11,
             verticalalignment='top',
             bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    plt.show()
    
    # 統計情報
    print("\n" + "=" * 60)
    print("Loss Statistics:")
    print(f"  Initial Loss: {trainer.loss_history[0]:.4f}")
    print(f"  Final Loss: {final_loss:.4f}")
    print(f"  Minimum Loss: {min_loss:.4f}")
    print(f"  Loss Reduction: {trainer.loss_history[0] - min_loss:.4f}")
    print(f"  Target (≤25.0): {'✅ Achieved!' if min_loss <= 25.0 else '⏳ Not yet - try more epochs or adjust hyperparameters'}")
    print("=" * 60)
else:
    print("No loss history recorded.")


## 10. モデルの保存


In [ ]:
# モデルの保存
import torch

save_dir = "./checkpoints/final_model"
os.makedirs(save_dir, exist_ok=True)

torch.save({
    "model_state_dict": model.state_dict(),
    "config": model_config,
}, os.path.join(save_dir, "model.pt"))

tokenizer.save_pretrained(save_dir)

print(f"Model saved to {save_dir}")


## 11. 推論テスト（オプション）


In [ ]:
# 学習済みモデルでテキスト生成をテスト
model.eval()

test_prompts = [
    "The quick brown fox",
    "In the beginning",
    "Machine learning is"
]

for prompt in test_prompts:
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    
    with torch.no_grad():
        generated = model.generate(
            input_ids,
            max_length=50,
            temperature=0.8,
            top_k=50,
            top_p=0.9
        )
    
    generated_text = tokenizer.decode(generated[0], skip_special_tokens=True)
    print(f"\nPrompt: {prompt}")
    print(f"Generated: {generated_text}")


## まとめ

このノートブックでは、RevDEQモデルを学習し、lossが25程度まで下がることを確認しました。

**主なポイント:**
- モデルサイズをColabで実用的な範囲に調整
- 複数エポックでの学習でlossを下げる
- Loss履歴を可視化して目標達成を確認
- 目標（loss ≤ 25）に達した場合は通知

**もしlossが25まで下がらない場合:**
- エポック数を増やす（`num_train_epochs`を増やす）
- 学習率を調整（`learning_rate`を変更）
- モデルサイズを大きくする（`hidden_size`を増やす）
- データセットサイズを増やす
